In [52]:
import os
import requests

def transcribe_file(file_path):
    url = "https://transcribe.whisperapi.com"
    headers = {
        "Authorization": f"Bearer {os.getenv('WHISPERAPI_TOKEN')}"
    }
    with open(file_path, "rb") as file:
        files = {"file": file}
        data = {
            "fileType": "mp4",
            "diarization": "false",
            "language": "en",
            "task": "transcribe"
        }

        response = requests.post(url, headers=headers, files=files, data=data)

    return response.json()["text"]

file_path = "/Users/blake/Desktop/lecture_thirteen.mp4"
transcription = transcribe_file(file_path)
print(transcription)

 I'm available, you know, I look at them. Yeah. What else? Anybody have any questions on this? There are a lot of websites that will make each timeline for you. Just, you know, if you really do pick one of those, just make sure it looks nice. Yeah. Let me, let me just wrap it over. Okay, and just to remind you of the example on the website at the 20 seconds. So lots of stuff becoming, uh, it's the wrong class. Here it is. So the last class I was talking about this, um, Transatlantic Slate Trade, and we went from this table comparing, um, the Transatlantic Slate Trade, uh, to the familial system that existed in Africa, uh, prior to the European's arriving. The Europeans really are just going to, to build on to the slave system that the African States already have. Um, but they really fundamentally changed the nature of that slavery. The European version of slavery is very different than what African peoples had experienced prior to them. And this, this is a very famous picture, uh, abou

In [60]:
import openai
import textwrap
import os

# Retrieve API key from environment variable
API_KEY = os.getenv('OPENAI_API_KEY')

def generate_corrected_transcripts(response_text):
    openai.api_key = API_KEY
    wrapped_text = textwrap.wrap(response_text, width=8000)

    corrected_chunks = map(lambda chunk: openai.ChatCompletion.create(
        model="gpt-3.5-turbo",
        messages=[
            {
                "role": "system",
                "content": f"""
                The following is a transcript taken from a Zoom lecture, however, there are some factual and grammatical errors in its transcription. Rewrite the transcript to remove any such errors. Keep the same tone of voice and perspective in the transcription.

                <<TRANSCRIPTION>>
                {chunk}

                Rewrite the transcript to remove any grammatical or factual errors:


                """
            }
        ]
    )['choices'][0]['message']['content'], wrapped_text)

    return list(corrected_chunks)

response = {"text": transcription}
corrected_transcripts = generate_corrected_transcripts(response["text"])


In [61]:
def extract_important_events(transcripts):
    important_events = map(lambda transcript: openai.ChatCompletion.create(
        model="gpt-3.5-turbo",
        messages=[
            {
                "role": "system",
                "content": f"""
                Pick out two of the most important events or things that happened from the following transcription, along with the time period/year in which they occurred.

                <<TRANSCRIPTION>>
                {transcript}

                Two most important events or things as well as their dates:


                """
            }
        ]
    )['choices'][0]['message']['content'], transcripts)

    return list(important_events)

important_events = extract_important_events(corrected_transcripts)


In [62]:
def generate_chronological_summary(events):
    events_text = '\n'.join(events)
    response = openai.ChatCompletion.create(
        model="gpt-3.5-turbo",
        messages=[
            {
                "role": "system",
                "content": f"""
                Given the set of events, order them in chronological order, giving a concise summary of the context of the event, the event, and the date in which the event occurred. All of the events should be distinct, so all events may not be used.

                <<TRANSCRIPTION>>
                {events_text}

                Chronology:


                """
            }
        ]
    )

    return response['choices'][0]['message']['content']

chronological_summary = generate_chronological_summary(important_events)


In [63]:
generated_code = openai.ChatCompletion.create(
    model="gpt-3.5-turbo",
    messages=[
            {"role": "system", "content": r"""
            Write the LaTeX code to turn the sequence of events below into a simple historical timeline. 
            Use the Tikz package. The timeline should include the name of the event, the date of the event, 
            and a very short description of the event. The timeline should look decent and have endpoints. 
            Please orient the timeline in landscape mode and use two rows to make it all fit on one page. 
            Very important that it fits on one page. Use the example below as an example of what the code 
            might look like.

            <<EXAMPLE CODE>>
            \documentclass{article}
            \usepackage[utf8]{inputenc}
            \usepackage{tikz}
            \usetikzlibrary{shapes,arrows.meta,calc,positioning}
            \usepackage{lscape}
            \usepackage[margin=1in]{geometry}

            \begin{document}
            \pagenumbering{gobble}

            \begin{landscape}
            \begin{center}
            \begin{tikzpicture}[x=0.6cm, y=1cm, font=\small]
            \tikzset{event/.style={rectangle, rounded corners, draw=black, fill=white, text centered, text width=3cm, minimum height=1cm}}
            \tikzset{arrow/.style={-Stealth,thick}}

            \node[event] (A) at (0,0) {1786\\ \textbf{French government faces bankruptcy}\\King Louis XVI attempts to raise taxes on the Second Estate};
            \node[event, right=1cm of A] (B) {May 1789\\ \textbf{Estates General meets}\\Conflicts between king and Third Estate};
            \node[event, right=1cm of B] (C) {May-June 1789\\ \textbf{Violent protests and riots}\\Formation of a militia to protect Paris};
            \node[event, right=1cm of C] (D) {June 17, 1789\\ \textbf{Formation of National Assembly}\\Represents not just commoners but whole of France};
            \node[event, right=1cm of D] (E) {June 20, 1789\\ \textbf{Tennis Court Oath}\\Third Estate swears not to separate until a constitution is established};

            \node[event, below=1cm of A] (F) {June 27, 1789\\ \textbf{King Louis XVI joins National Assembly}\\End of absolute monarchy in France};
            \node[event, right=1cm of F] (G) {August 4, 1789\\ \textbf{Abolition of Feudal Regime}\\Nobles and clergy renounce privileges};
            \node[event, right=1cm of G] (H) {End of August 1789\\ \textbf{Declaration of the Rights of Man}\\Outlines basic liberties and rights};
            \node[event, right=1cm of H] (I) {1792\\ \textbf{Mary Wollstonecraft's "A Vindication of the Rights of Woman"}\\Challenges traditional views on women's education};
            \node[event, right=1cm of I] (J) {1790\\ \textbf{Catherine Macaulay's "Letters on Education"}\\Argues for importance of education, including for women};

            \draw[arrow] (A) -- (B);
            \draw[arrow] (B) -- (C);
            \draw[arrow] (C) -- (D);
            \draw[arrow] (D) -- (E);
            \draw[arrow] (E) |- ($(E.south east) + (0.5, -0.5)$) -| (F);
            \draw[arrow] (F) -- (G);

            <<SEQUENCE OF EVENTS>>
            
            """+"""
            {}
            
            
            Code:
            
            """.format(chronological_summary)}
      ]
)['choices'][0]['message']['content']

print(generated_code)

\documentclass{article}
            \usepackage[utf8]{inputenc}
            \usepackage{tikz}
            \usetikzlibrary{shapes,arrows.meta,calc,positioning}
            \usepackage{lscape}
            \usepackage[margin=1in]{geometry}

            \begin{document}
            \pagenumbering{gobble}

            \begin{landscape}
            \begin{center}
            \begin{tikzpicture}[x=0.6cm, y=1cm, font=\small]
            \tikzset{event/.style={rectangle, rounded corners, draw=black, fill=white, text centered, text width=3cm, minimum height=1cm}}
            \tikzset{arrow/.style={-Stealth,thick}}

            \node[event] (A) at (0,0) {1483\\ \textbf{Congo-Portugal Relations}\\Portugal initiates contact with Congo};
            \node[event, right=1cm of A] (B) {1500\\ \textbf{Slave Trade Grows}\\Congo-Portugal trade shifts toward acquisition of slaves};
            \node[event, right=1cm of B] (C) {\textbf{Transatlantic Slave Trade Begins}\\Acquisition of slaves fuels growing s